<a href="https://colab.research.google.com/github/sayedmohamedscu/improve-OCT-scan-using-CNN/blob/master/oct_new_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir .kaggle
!mkdir .s
#os.chdir("/content/gdrive/My Drive/")

In [0]:
import json
token = {"username":"tesla2","key":"2e88d384a9e1fa7950bfae00ad07e151"} #use your own you will find repo i explain how to connect kaggle to colab
with open("/content/.s/kaggle.json", "w") as file:
    json.dump(token, file)
!ls

87.oct-in-cnv.01.intro.fig.2.jpg  gdrive		     sample_data
Findings-Classic-CNV-C.jpg	  RP_November_A08_Fig02.jpg


In [0]:
!chmod 600 /content/.s/kaggle.json

In [0]:
!cp /content/.s/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!kaggle datasets download -d paultimothymooney/kermany2018 -p /content 

100% 5.41G/5.42G [00:56<00:00, 79.6MB/s]
100% 5.42G/5.42G [00:56<00:00, 104MB/s] 


In [0]:
!unzip -qq /content/kermany2018.zip

In [0]:
!unzip -qq /content/OCT2017.zip

warning [/content/OCT2017.zip]:  4294967296 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  4294967296
  (attempting to re-compensate)
file #122249:  bad zipfile offset (local header sig):  51410
  (attempting to re-compensate)


In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator



train_datagen = ImageDataGenerator(
      rescale=1./255)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        '/content/OCT2017 /train',
        # All images will be resized to 240*240
        target_size=(224, 224),
        batch_size=100,
        shuffle=True,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/OCT2017 /test',
        
        target_size=(224, 224),
        batch_size=50,
        shuffle=True,
        class_mode='categorical')

In [0]:
import keras
from keras import models
from keras import layers

from keras.layers import Dense,GlobalAveragePooling2D,Flatten,Dropout,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D




In [0]:
from keras.applications import VGG16
from keras.applications import MobileNet
from keras.applications.inception_v3 import InceptionV3


base_model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))
base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____

In [0]:
x=base_model.output
from keras.models import Model
x = Conv2D(512, kernel_size = (3,3), padding = 'valid')(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
#x = keras.layers.GlobalAveragePooling2D()(x)
x = Flatten()(x)
x=Dropout(0.2)(x)


x = Dense(50, activation='relu')(x)

x = Dense(4 , activation='sigmoid')(x)
model=Model(inputs=base_model.input,outputs=x)
model.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________

In [0]:
model.trainable = True

set_trainable = False
for layer in model.layers:
    if layer.name == 'conv2d_1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
from keras import models
from keras import layers
from keras import optimizers
import keras

from keras import backend as K


In [0]:
import os

os.getcwd()
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir("/content/gdrive/My Drive/")
!ls

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0005),
              metrics=['acc'])



In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=840,
      epochs=15,
      
      validation_data=validation_generator,
      validation_steps=20)
model.save('/content/gdrive/My Drive/occt.h5')


Instructions for updating:
Use tf.cast instead.
Epoch 1/15
840/840 [==============================] - 607s 722ms/step - loss: 0.4966 - acc: 0.8056 - val_loss: 0.2025 - val_acc: 0.9236
Epoch 2/15
840/840 [==============================] - 552s 657ms/step - loss: 0.2927 - acc: 0.8966 - val_loss: 0.1172 - val_acc: 0.9607
Epoch 3/15
840/840 [==============================] - 533s 635ms/step - loss: 0.2475 - acc: 0.9128 - val_loss: 0.2322 - val_acc: 0.8936
Epoch 4/15
840/840 [==============================] - 512s 610ms/step - loss: 0.2209 - acc: 0.9217 - val_loss: 0.0942 - val_acc: 0.9762
Epoch 5/15
840/840 [==============================] - 502s 598ms/step - loss: 0.2013 - acc: 0.9279 - val_loss: 0.1022 - val_acc: 0.9597
Epoch 6/15
840/840 [==============================] - 487s 580ms/step - loss: 0.1824 - acc: 0.9348 - val_loss: 0.1113 - val_acc: 0.9659
Epoch 7/15
840/840 [==============================] - 486s 579ms/step - loss: 0.1681 - acc: 0.9402 - val_loss: 0.0883 - val_acc: 0.9762


In [0]:
del model

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# dimensions of our images
img_width, img_height = 224, 224

# load the model we saved
model = load_model('/content/gdrive/My Drive/occt.h5')

# predicting images
img = image.load_img('/content/bjophthalmol-2014-July-98-Suppl 2-ii24-F1.large.jpg', target_size=(img_width, img_height))

x = image.img_to_array(img)
print(x.dtype)


x=x/255
x= x.reshape(1, 224, 224, 3)


classes = model.predict(x)

print (np.argmax(classes))
print ((classes))

float32
(1, 224, 224, 3)
1
[[0.         0.99999964 0.         0.        ]]
